# Data Cleaning - Twitter

In [1]:
#import libraries

import pandas as pd
import numpy as np
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import datetime

import pickle

In [2]:
dt_tweets_raw = pd.read_csv('data/trump_tweets.txt')

In [3]:
pd.set_option('display.max_colwidth',-1)  

In [4]:
dt_tweets = dt_tweets_raw.drop(['source', 'retweet_count', 'favorite_count', 'is_retweet', 'id_str'], axis=1)

In [8]:
dt_tweets = dt_tweets.set_index('created_at')

**Cleaning tweets**

In [9]:
dt_tweets['text'] = dt_tweets['text'].apply(lambda x: clean_tweet(x))

NameError: name 'clean_tweet' is not defined

In [ ]:
#Change date object to datetime
dt_tweets['created_at'] = pd.to_datetime(dt_tweets['created_at'])

In [7]:
#create date column
dt_tweets['date'] = [d.date() for d in dt_tweets['created_at']]

AttributeError: 'str' object has no attribute 'date'

In [199]:
#drop created by column 
dt_tweets = dt_tweets.drop(['created_at'], axis=1)

In [200]:
#Change date object to datetime
dt_tweets['date'] = pd.to_datetime(dt_tweets['date'])

In [201]:
#make dates the index
#dt_tweets = dt_tweets.set_index('date')

In [202]:
dt_tweets.head(10);

**Sentiment Analysis**

In [203]:
analyzer = SentimentIntensityAnalyzer()
sentiment = dt_tweets['text'].apply(lambda x: analyzer.polarity_scores(x))

In [204]:
dt_tweets=pd.concat([dt_tweets,sentiment.apply(pd.Series)],1)

In [10]:
dt_tweets.head(10)

,text
created_at,
01-23-2017 11:38:16,Busy week planned with a heavy focus on jobs and national security. Top executives coming in at 9:00 A.M. to talk manufacturing in America.
01-24-2017 11:11:47,Will be meeting at 9:00 with top automobile executives concerning jobs in America. I want new plants to be built here for cars sold here!
01-24-2017 16:58:06,A photo delivered yesterday that will be displayed in the upper/lower press hall. Thank you Abbas! https://t.co/Uzp0ivvRp0
01-24-2017 17:04:01,Great meeting with automobile industry leaders at the @WhiteHouse this morning. Together we will #MAGA! https://t.co/OXdiLOkGsZ
01-24-2017 17:49:17,Signing orders to move forward with the construction of the Keystone XL and Dakota Access pipelines in the Oval Off… https://t.co/aOxmfO0vOK
01-25-2017 00:46:57,Great meeting with Ford CEO Mark Fields and General Motors CEO Mary Barra at the @WhiteHouse today. https://t.co/T0eIgO6LP8
01-25-2017 02:16:19,Congratulations to @FoxNews for being number one in inauguration ratings. They were many times higher than FAKE NEWS @CNN - public is smart!
01-25-2017 02:25:40,"If Chicago doesn't fix the horrible ""carnage"" going on 228 shootings in 2017 with 42 killings (up 24% from 2016) I will send in the Feds!"
01-25-2017 02:37:48,Big day planned on NATIONAL SECURITY tomorrow. Among many other things we will build the wall!


**Categorizing Tweets**

In [206]:
dt_tweets['sentiment'] = np.where(dt_tweets['compound']<0, 1, 0)

In [207]:
dt_tweets.groupby('sentiment').size()

sentiment
0    4299
1    1805
dtype: int64

In [208]:
dt_tweets.tail(5)

,text,date,neg,neu,pos,compound,sentiment
6099,RT By the time their fake news campaigns are fully exposed they ve already moved on to their next fake news campaign A g,2019-01-19,0.291,0.709,0.000,-0.7778,1
6100,Remember it was Buzzfeed that released the totally discredited Dossier paid for by Crooked Hillary Clinton and the Democrats as opposition research on which the entire Russian probe is based A very sad day for journalism but a great day for our Country,2019-01-19,0.090,0.793,0.118,0.5199,0
6101,RT At what point in fairness after 2 years do Americans of good will say enough already If the SpecialCounsel had collusi,2019-01-19,0.000,0.873,0.127,0.4404,0
6102,RT This is just the most egregious example of the rampant unfairness that has tainted this partisan witch hunt from the beg,2019-01-19,0.106,0.894,0.000,-0.3612,1
6103,Fake News is truly the ENEMY OF THE PEOPLE,2019-01-19,0.452,0.370,0.179,-0.6633,1


In [209]:
#save dataframe as pickle file

with open('dt_tweets.pickle', 'wb') as to_write:
    pickle.dump(dt_tweets, to_write)